[![Open In Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/tushar-mahalya/Forecasting-using-Bidirectional-LSTM/blob/master/model_selector.ipynb)

In [6]:
# Importing important pakages
import os
import json
import numpy as np
import pandas as pd
from collections import OrderedDict
from termcolor import colored as color

# Importing TF.keras packages
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Dense, LSTM, Dropout, Bidirectional

# To avoid irrelevent warning messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2023-04-05 04:41:06.114471: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Yahoo Finance API
import yfinance

# Function to download stock data (univariate or multivariate)
from Data_Downloader import data_accumilator

In [8]:
# Importing custom defined models into local runtime
from Models import models
my_models = models()
my_models

[('Fixed', <function Models.simple_model(X, y, learn_rate)>),
 ('Dynamic', <function Models.dynamic_model(X, y, learn_rate)>),
 ('Bidirectional', <function Models.bidirectional_model(X, y, learn_rate)>),
 ('Stacked', <function Models.stacked_model(X, y, learn_rate)>)]

In [9]:
# Nifty50 stocks indormation
nifty50 = pd.read_csv('nifty50.csv')
nifty50.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Enterprises Ltd.,Metals & Mining,ADANIENT,EQ,INE423A01024
1,Adani Ports and Special Economic Zone Ltd.,Services,ADANIPORTS,EQ,INE742F01042
2,Apollo Hospitals Enterprise Ltd.,Healthcare,APOLLOHOSP,EQ,INE437A01024
3,Asian Paints Ltd.,Consumer Durables,ASIANPAINT,EQ,INE021A01026
4,Axis Bank Ltd.,Financial Services,AXISBANK,EQ,INE238A01034


In [10]:
# Get the current working directory (cwd)
cwd = os.getcwd()

# Downloading Nifty50 stocks data in Current working directory
preprocessed_data = data_accumilator(cwd, nifty50['Symbol'].sample(2))

Directory named "Stocks Data" with required data is downloaded successfully at "/home/studio-lab-user/sagemaker-studiolab-notebooks/Forecasting-using-Bidirectional-LSTM"


In [36]:
# Dictionary with stock ticker(key) and location of 
# normalized data of that stock (value)
dict(list(preprocessed_data.items())[:5])  # First 5 elements

{'ADANIENT': '/home/studio-lab-user/sagemaker-studiolab-notebooks/Forecasting-using-Bidirectional-LSTM/Stocks Data/ADANIENT/Normalized_ADANIENT.csv',
 'ADANIPORTS': '/home/studio-lab-user/sagemaker-studiolab-notebooks/Forecasting-using-Bidirectional-LSTM/Stocks Data/ADANIPORTS/Normalized_ADANIPORTS.csv',
 'APOLLOHOSP': '/home/studio-lab-user/sagemaker-studiolab-notebooks/Forecasting-using-Bidirectional-LSTM/Stocks Data/APOLLOHOSP/Normalized_APOLLOHOSP.csv',
 'ASIANPAINT': '/home/studio-lab-user/sagemaker-studiolab-notebooks/Forecasting-using-Bidirectional-LSTM/Stocks Data/ASIANPAINT/Normalized_ASIANPAINT.csv',
 'AXISBANK': '/home/studio-lab-user/sagemaker-studiolab-notebooks/Forecasting-using-Bidirectional-LSTM/Stocks Data/AXISBANK/Normalized_AXISBANK.csv'}

In [11]:
# Importing data initializing function
from Data_Initializer import train_test_data

# Importing best model selector function
from Model_Selector import test_models

In [17]:
res = test_models_dumdum(preprocessed_data, 2, my_models, window_sizes=[2])


================================ NSE : CIPLA ================================

--------------------------------------------------------------- Simple Sequence

Window size : 2
----------------
  > Model: Fixed           Param count: 146 	Training error: 0.0958	Testing error: 0.4501
  > Model: Dynamic         Param count: 35 	Training error: 0.0364	Testing error: 0.1672
  > Model: Bidirectional   Param count: 77 	Training error: 0.0813	Testing error: 0.4850
  > Model: Stacked         Param count: 806 	Training error: 0.0793	Testing error: 0.4774

--------------------------------------------------------------- Multi Sequence

Window size : 2
----------------
  > Model: Fixed           Param count: 186 	Training error: 0.1469	Testing error: 0.8378
  > Model: Dynamic         Param count: 51 	Training error: 0.2217	Testing error: 1.2661
  > Model: Bidirectional   Param count: 109 	Training error: 0.0786	Testing error: 0.3817
  > Model: Stacked         Param count: 886 	Training error: 0.08